#### Data Consistency Check

In [456]:
# Import libraries & Declare the path to the data files
import pandas as pd
import numpy as np
import os
path = r'C:\Users\PC\Desktop\CareerFoundry Data Analytics\08-2025 Instacart Basket Analysis\02 Data\Original Data'
path_data = r'C:\Users\PC\Desktop\CareerFoundry Data Analytics\08-2025 Instacart Basket Analysis'

In [457]:
# Import data 
df_ord = pd.read_csv(os.path.join(path, 'orders.csv'), index_col=False)
df_prods = pd.read_csv(os.path.join(path, 'products.csv'), index_col=False)
df_dep = pd.read_csv(os.path.join(path, 'departments.csv'), index_col = False)
df_ord = df_ord.drop(columns = ['eval_set'])

In [458]:
# Find another identifier variable in the df_ords dataframe that doesn’t need to be included in your analysis as a numeric variable and change it to a suitable format.
df_ord['user_id'] = df_ord['user_id'].astype('str')

In [459]:
# Look for a variable in your df_ords dataframe with an unintuitive name and change its name without overwriting the dataframe.
df_ord.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

In [460]:
#### df_prods

In [461]:
#### 1.1 Check for mixed values 

In [462]:
for col in df_prods.columns.tolist():
  weird = (df_prods[[col]].map(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_prods[weird]) > 0:
    print (col)

product_name


In [463]:
#df_prods['product_name'] = df_prods['product_name'].astype('str') - I tried to change the type of the mixed values to string, but if I do so, I can't identify the missing values later

In [464]:
#### 1.2 Check for missing values

In [465]:
df_prods.isnull().sum()

# Create a new dataframe, df_nan, containing on the values within the 'product_name' column that meets the condition of 'isnull()' true
df_nan = df_prods[df_prods['product_name'].isnull() == True]
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [466]:
df_nan.shape

(16, 5)

In [467]:
# Create a new dataframe, setting the 'isnull()' condition to False so that only non-missing values are present
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]
df_prods_clean.shape

(49677, 5)

In [468]:
#### 1.3 Check for duplicates in the cleaned df 

In [469]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [470]:
# Remove the duplicate values 
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()
df_prods_clean_no_dups.shape

(49672, 5)

In [471]:
df_prods['product_name'] = df_prods['product_name'].astype('str')

In [472]:
#### 1.4 Export the cleaned and updated version of the df 

In [473]:
df_prods_clean_no_dups.to_csv(os.path.join(path_data, '02 Data','Prepared Data', 'products_cleaned.csv'))

In [474]:
#### df_ord

In [475]:
df_ord.describe()

,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


order_number: max=100 - possibly high for dataset timeframe
orders_day_of_week: 0–6 - zero-indexed (0 = Sunday or Monday)
days_since_prior_order: max=30 - possible monthly cap, min=0 - same-day orders or first-time customers, fewer entries - missing data likely for new customers
order_id, order_number - identifiers - store as strings

In [476]:
#### 2.1 Check for mixed-type data in your df_ords dataframe.

In [477]:
df_ord.dtypes

order_id                    int64
user_id                    object
order_number                int64
orders_day_of_week          int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

In [478]:
for col in df_ord.columns.tolist():
  weird = (df_ord[[col]].map(type) != df_ord[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ord[weird]) > 0:
    print (col)

There are no mixed-type columns in this df. 

In [479]:
#### 2.2 Check for missing values in df_ord

In [480]:
df_ord.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

There are 206209 missing values in the days_since_prior_order column. These likely represent customers’ first orders, so imputation is not necessary.

In [481]:
# Address the missing values using an appropriate method
# Separate missing values in a new df  
df_ord_nan = df_ord[df_ord['days_since_prior_order'].isnull() == True]
df_ord_nan

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,NaN
3420934,3189322,206206,1,3,18,NaN
3421002,2166133,206207,1,6,19,NaN
3421019,2227043,206208,1,1,15,NaN


In [482]:
# Filter the DataFrame where 'order_number' equals 1 and check if 'days_since_prior_order' is NaN in all rows
df_first_orders = df_ord_nan[df_ord_nan['order_number'] == '1']
df_first_orders_missing = df_first_orders['days_since_prior_order'].isna().all()
df_first_orders_missing

np.True_

In [483]:
#### REVISED 

In [484]:
# Create a new boolean column, which will be True if NaN and False if not.
df_ords_clean = df_ord  
df_ords_clean['new_customer'] = df_ord['days_since_prior_order'].isnull() == True 

In [485]:
#### 2.3 Run a check for duplicate values in your df_ords data

In [486]:
# df_ords_clean = df_ord[df_ord.duplicated()]
df_ords_clean_no_dups = df_ords_clean.drop_duplicates()
df_ords_clean_no_dups.shape

(3421083, 7)

There are no duplicate values in df_ord. Potential duplicates, if present, can be removed using .drop_duplicates().

In [487]:
df_ords_clean_no_dups['order_id'] = df_ords_clean_no_dups['order_id'].astype('str')
df_ords_clean_no_dups['order_number'] = df_ords_clean_no_dups['order_number'].astype('str')

#### 2.4 Export the df

In [488]:
df_ords_clean_no_dups.to_csv(os.path.join(path_data, '02 Data','Prepared Data', 'orders_cleaned.csv'))